# A simple benders example

In [1]:
import calliope
import pandas as pd
import pyomo.kernel as pmo

## Setup energy system models using Calliope

In [ ]:
config = "national_scale/model.yaml"
first_hour = "2005-01-01T00:00"

model = {
    "main": {"calliope": calliope.Model(config, override_dict={"config.init.time_subset": [first_hour, first_hour]})},
    "sub": {"calliope": calliope.Model(config)},
}

for k in model.keys():
    model[k]["calliope"].build()
    model[k]["T"] = len(model[k]["calliope"].inputs.timesteps)
    model[k]["backend"] = model[k]["calliope"].backend
    model[k]["pyomo"] = model[k]["backend"]._instance
    model[k]["vars"] = model[k]["backend"].variables.to_dataframe()

## Prepare linking variables

In [3]:
link_var_names = ["flow_cap", "storage_cap", "link_flow_cap", "area_use"]
link_var_names = [vn for vn in model["main"]["vars"].columns if vn in link_var_names]
link_vars = [
    (name, i) for name in link_var_names for (i, el) in model["main"]["vars"][name].items() if not pd.isnull(el)
    ]

## Modify main-model

In [4]:
# Deactivate operational variables.
for name in model["main"]["vars"].columns:
    if name in link_var_names:
        continue
    for var in model["main"]["vars"][name].dropna().values:
        var.deactivate()

# Prepare cut constraints and approximation variable (theta).
model["main"]["pyomo"].c_cuts = pmo.constraint_list()
model["main"]["pyomo"].v_theta = pmo.variable(domain=pmo.NonNegativeReals)

# Prepare objective.
total_invest_cost = model["main"]["backend"].get_global_expression("cost_investment_annualised").sum().item()
model["main"]["backend"].objectives.get("min_cost_optimisation").item().deactivate()
model["main"]["pyomo"].e_cost = total_invest_cost * model["sub"]["T"] / model["main"]["T"]
model["main"]["pyomo"].obj = pmo.objective(model["main"]["pyomo"].e_cost + model["main"]["pyomo"].v_theta)

### Adding bounds to main-model variables

Try not executing this cell and see how this affects the convergence.

In [5]:
# Ensure that linking variables are bounded.
for name, idx in link_vars:
    x = model["main"]["vars"].loc[idx, name]
    x.lb = x.lb or 0.0
    x.ub = x.ub or 1e6

## Modify sub-model

In [6]:
# Fixing linked variables, including penalized slack variables (to ensure feasibility).
model["sub"]["pyomo"].p_link = pmo.parameter_dict()
model["sub"]["pyomo"].c_link = pmo.constraint_dict()
model["sub"]["pyomo"].v_slack_pos = pmo.variable_dict()
model["sub"]["pyomo"].v_slack_neg = pmo.variable_dict()

for name, idx in link_vars:
    model["sub"]["pyomo"].p_link[(name, idx)] = pmo.parameter(0.0)
    model["sub"]["pyomo"].v_slack_pos[(name, idx)] = pmo.variable(domain=pmo.NonNegativeReals)
    model["sub"]["pyomo"].v_slack_neg[(name, idx)] = pmo.variable(domain=pmo.NonNegativeReals)
    model["sub"]["pyomo"].c_link[(name, idx)] = pmo.constraint(
        model["sub"]["pyomo"].p_link[(name, idx)]
        + model["sub"]["pyomo"].v_slack_pos[(name, idx)]
        - model["sub"]["pyomo"].v_slack_neg[(name, idx)]
        == model["sub"]["vars"].loc[idx, name]
    )

# Prepare objective.
model["sub"]["backend"].objectives.get("min_cost_optimisation").item().deactivate()
model["sub"]["pyomo"].obj = pmo.objective(
    model["sub"]["backend"].get_global_expression("cost_operation_variable").sum().item()
    + 1e5 * (sum(model["sub"]["pyomo"].v_slack_pos.values()) + sum(model["sub"]["pyomo"].v_slack_neg.values()))
)

## Prepare optimization

In [7]:
# Optimizer for main model.
opt_m = pmo.SolverFactory("gurobi")
opt_m.options["Method"] = 2
opt_m.options["Crossover"] = 0

# Optimizer for sub model (make sure duals are extracted).
opt_s = pmo.SolverFactory("gurobi")
model["sub"]["pyomo"].dual.activate()

# Track best bounds.
lb, ub = -1e100, +1e100

## Start iteration

In [ ]:
print("  iter  |  lb         |  ub         |  gap")
print("--------|-------------|-------------|-------------")
for iter in range(150):
    # Optimize main.
    ret = opt_m.solve(model["main"]["pyomo"])

    # Fix linking vars.
    for name, idx in link_vars:
        model["sub"]["pyomo"].p_link[(name, idx)].value = model["main"]["vars"].loc[idx, name].value or 0.0

    # Optimize sub.
    ret = opt_s.solve(model["sub"]["pyomo"])
    obj_val_sub = pmo.value(model["sub"]["pyomo"].obj)

    # Update bounds.
    lb = max(lb, pmo.value(model["main"]["pyomo"].obj.expr))
    ub = min(ub, pmo.value(model["main"]["pyomo"].e_cost) + obj_val_sub)

    # Update gap, check termination, and log.
    gap = abs((ub - lb) / ub)
    if (iter % 5) == 0:
        print(f"  {iter:4d}  |  {lb:.3e}  |  {ub:.3e}  |  {gap:.1e}")
    if gap < 1e-2:
        print(f"Reached gap of {gap:.1e} after {iter + 1} iterations, terminating.")
        break

    # Build cut.
    cut_rhs = obj_val_sub
    for name, idx in link_vars:
        x = model["main"]["vars"].loc[idx, name]
        dual = model["sub"]["pyomo"].dual.get(model["sub"]["pyomo"].c_link[(name, idx)])
        cut_rhs -= dual * (x - x.value)

    # Add cut.
    model["main"]["pyomo"].c_cuts.append(pmo.constraint(model["main"]["pyomo"].v_theta >= cut_rhs))